# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [ ]:
pip install python-dotenv


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

price_data_path=os.getenv("PRICE_DATA")

print(f"PRICE_DATA: {price_data_path}")




In [4]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [ ]:
from dotenv import load_dotenv
import os
import glob


parquet_files = glob.glob(os.path.join(price_data_path, '**/*.parquet'), recursive=True)


print("Found parquet files:")
for file in parquet_files:
    print(file)


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
import os
import glob
import dask.dataframe as dd
from dotenv import load_dotenv


load_dotenv()
price_data_dir = os.getenv('PRICE_DATA')

parquet_files = glob.glob(os.path.join(price_data_dir, '**', '*.parquet'), recursive=True)


parquet_file_paths = []
for path in parquet_files:
    if os.path.isfile(path):  
        parquet_file_paths.append(path)


ddf = dd.read_parquet(parquet_file_paths, recursive=True)


ddf = ddf.sort_values(by=['Date'])
ddf['Close_lag'] = ddf.groupby('Date')['Close'].shift(1)
ddf['Adj_Close_lag'] = ddf.groupby('Date')['Adj Close'].shift(1)
ddf['returns'] = (ddf['Adj Close'] / ddf['Adj_Close_lag']) - 1
ddf['hi_lo_range'] = ddf['High'] - ddf['Low']

dd_feat = ddf

result = dd_feat.compute()


print(result.head())



In [9]:
pip install --upgrade pyarrow


Note: you may need to restart the kernel to use updated packages.


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [ ]:
import dask.dataframe as dd
import pandas as pd


df=ddf.compute()
df['Rolling_Avg_Return']=df['returns'].rolling(window=10).mean()
print(df.head())




Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

It wasn't necessary to convert to Pandas to calculate the moving average return. Doing it directly in Dask would have been better because Dask operates efficiently on large datasets that don't fit into memory. Dask's .rolling() and .mean() methods work similarly to Pandas but are optimized to handle distributed computing, allowing for scalable calculations across multiple cores or even clusters without the limitations of in-memory operations. This approach ensures better performance and scalability, especially when dealing with big data.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.